In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from gerrychain import Graph, GeographicPartition, Election, MarkovChain, tree, constraints
from functools import partial
from gerrychain.updaters import Tally, cut_edges
from gerrychain.metrics import polsby_popper
from gerrychain.proposals import recom
import os
from gerrychain.accept import always_accept

from combination import *
from candidate import Candidate
from ballot import Ballot

In [4]:
PROCESSED_SHAPEFILE_PATH = os.path.join(os.getcwd())
NC_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "north carolina", "precinct_level_w_election.shp")
EXPORT_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "north carolina")
ITERATIONS = 1
FOUR_FOUR_FIVE = 3.25
TOTAL_DISTRICTS = 13


In [5]:
nc_graph = Graph.from_file(NC_PATH, ignore_errors=True)

/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)


In [6]:
total_population = sum(nc_graph.nodes[node]["TOTPOP"] for node in nc_graph.nodes)


In [7]:
partition_445 = tree.recursive_tree_part(nc_graph, [1, 2, 3], total_population / FOUR_FOUR_FIVE, "TOTPOP", 0.01, 1)

nc_election = Election("CongressionalRace" , {"Democratic": "D_VOTES", "Republican": "R_VOTES"})

In [8]:
new_partition445 = GeographicPartition(nc_graph, assignment=partition_445, updaters={
        "population": Tally("TOTPOP", alias="population"),
        "white": Tally("NH_WHITE", alias="white"),
        "black": Tally("NH_BLACK", alias="black"),
        "asian": Tally("NH_ASIAN", alias="asian"),
        "hisp": Tally("HISP", alias="hisp"),
        "other": Tally("NH_OTHER", alias="other"),
        "cut_edges": cut_edges,
        "CongressionalRace": nc_election
    })

In [9]:
ideal_pop445 = total_population / FOUR_FOUR_FIVE
proposal445 = partial(recom, pop_col="TOTPOP", pop_target=ideal_pop445, epsilon=.02, node_repeats=2)
compactness445 = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(new_partition445["cut_edges"])
)
pop_constraint445 = constraints.within_percent_of_ideal_population(new_partition445, 0.3)


In [10]:
chain445 = MarkovChain(
    proposal=proposal445,
    constraints=[
        compactness445,
        pop_constraint445
    ],
    accept=always_accept,
    initial_state=new_partition445,
    total_steps=ITERATIONS
)

In [ ]:
import random 

ballot_map = {}
vote_map = {}
all_candidates_dem = [Candidate("Adams, Alma S", "Democratic"), Candidate("Butterfield, G.K.", "Democratic"),
                     Candidate("Manning, Kathy E.", "Democratic"), Candidate("Price, David E.", "Democratic"), 
                     Candidate("Ross, Deborah K.", "Democratic"), Candidate("Zimmerman, Jeff", "Democratic"),
                     Candidate("Bridenstine, Jim", "Democratic"), Candidate("Carter, Earl", "Democratic"),
                     Candidate("Cohen, Steve", "Democratic"), Candidate("Cramer, Kevin", "Democratic"),
                     Candidate("Cummings, Elijah E.", "Democratic"), Candidate("Davis, Rodney", "Democratic"),
                     Candidate("DeLauro, Rosa L.", "Democratic") ]
                     
all_candidates_rep = [Candidate("Bishop, Dan", "Republican"), 
                     Candidate("Budd, Ted", "Republican"), Candidate("Cawthorn, Madison", "Republican"), 
                     Candidate("Foxx, Virginia", "Republican"), Candidate("Hudson, Richard", "Republican"), 
                     Candidate("McHenry, Patrick T.", "Republican"), Candidate("Murphy, Gregory", "Republican"), 
                     Candidate("Rouzer, David", "Republican"),
                     Candidate("Stevens, Mark", "Republican"), Candidate("Womack, Steve", "Republican"),
                     Candidate("Wright, Ron", "Republican"), Candidate("Yoder, Kevin", "Republican"),
                     Candidate("Zengerle, Mark", "Republican")]


for step in chain445:
    for part in step.parts:
        ballots = []

        total_reps = round((step["population"][part] / total_population) * TOTAL_DISTRICTS)
        dem_candidates = []
        rep_candidates = []
        for i in range(total_reps):
            dem_candidates.append(all_candidates_dem.pop())
            rep_candidates.append(all_candidates_rep.pop())

        rep_votes = 0
        dem_votes = 0
        for node in step.parts[part]:
            random.shuffle(dem_candidates)
            random.shuffle(rep_candidates)
            # Create ballots for the Republican and Democratic candidates
            ballots += [Ballot([rep for rep in rep_candidates]) for _ in range(round(nc_graph.nodes[node]["R_VOTES"]))]
            ballots += [Ballot([dem for dem in dem_candidates]) for _ in range(round(nc_graph.nodes[node]["D_VOTES"]))]
            rep_votes += round(nc_graph.nodes[node]["R_VOTES"])
            dem_votes += round(nc_graph.nodes[node]["D_VOTES"])
        ballot_map[part] = ballots
        vote_map[part] = (dem_votes + rep_votes, total_reps)


In [19]:
from collections import Counter

election_results_uneven = {}
initial_votes_uneven = {}

for district in ballot_map:
    ballots = ballot_map[district]
    winning_threshold = vote_map[district][0] / (1+vote_map[district][1])
    vote_counts = Counter()

    remaining_candidates = []
    eliminated_candidates = []
    winners = []
    winner_count = 0
    candidates_dict = {}
    rounds = 0

    for ballot in ballots:
        first_candidate = ballot.rankings[0]
        vote_counts[first_candidate] += ballot.weight

    remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2) + 2)
    remaining_candidates.sort(key=lambda x: x[1], reverse=True)
    initial_votes_uneven[district] = remaining_candidates

    while(True):
        remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2) - rounds - winner_count + 2)
        remaining_candidates.sort(key=lambda x: x[1], reverse=True)

        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            winners += remaining_candidates
            eliminated_candidates.append(winners.pop())
            break

        # SURPLUS ROUND
        first_place = remaining_candidates[0]
        if first_place[1] >= winning_threshold:
            remaining_candidates.remove(first_place)
            surplus = first_place[1] - winning_threshold

            sp_count = 0
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    ballot.rankings.remove(ballot.rankings[0])
                    ballot.set_weight(ballot.get_weight() * .8)
                    vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                    sp_count += 1
                    if sp_count >= surplus:
                        break
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    vote_counts[ballot.rankings[0]] -= 1

            first_place = (first_place[0], winning_threshold)
            winners.append(first_place) 
            rounds +=1
            winner_count += 1
            continue

        # ELIMINATION ROUND
        eliminated = remaining_candidates[(vote_map[district][1] * 2) - rounds-1 + 2]
        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            remaining_candidates.remove(eliminated)
            winners += remaining_candidates
            eliminated_candidates.append(eliminated)
            break
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                ballot.rankings.remove(ballot.rankings[0])
                ballot.set_weight(ballot.get_weight() * .8)
                vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                vote_counts[ballot.rankings[0]] -= 1

        eliminated_candidates.append(eliminated)
        rounds += 1
        election_results_uneven[district] = {"Winners": winners, "Eliminated": eliminated_candidates, "Initial Votes": initial_votes_uneven[district]}


In [20]:
# make the candidates json serializable
for district in election_results_uneven:
    winners = election_results_uneven[district]["Winners"]
    eliminated = election_results_uneven[district]["Eliminated"]
    initial_votes = election_results_uneven[district]["Initial Votes"]
    election_results_uneven[district]["RepCountTotal"] = vote_map[district][1]
    election_results_uneven[district]["VoteThreshold"] = vote_map[district][0] / (1+vote_map[district][1])
    election_results_uneven[district]["Winners"] = [(winner[0].name, winner[0].party, winner[1]) for winner in winners]
    election_results_uneven[district]["Eliminated"] = [(eliminated[0].name, eliminated[0].party, eliminated[1]) for eliminated in eliminated]
    election_results_uneven[district]["Initial Votes"] = [(initial_votes[0].name, initial_votes[0].party, initial_votes[1]) for initial_votes in initial_votes]


In [21]:
import json
# Export the results to a JSON file
with open('nc_election_results.json', 'w') as fp:
    json.dump(election_results_uneven, fp)
    
